In [1]:
# Uncomment to run the notebook in Colab
# ! pip install -q "wax-ml[complete]@git+https://github.com/eserie/wax-ml.git"
# ! pip install -q --upgrade jax jaxlib==0.1.67+cuda111 -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [ ]:
# check available devices
import jax

In [2]:
print("jax backend {}".format(jax.lib.xla_bridge.get_backend().platform))
jax.devices()

# 🎛 The 3-steps workflow 🎛

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eserie/wax-ml/blob/main/docs/notebooks/04_The_three_steps_workflow.ipynb)

It is already very useful to be able to execute a JAX function on a dataframe in a single work step
and with a single command line thanks to WAX-ML accessors.

The 1-step WAX-ML's stream API works like that:
```python
<data-container>.stream(...).apply(...)
```

But this is not optimal because, under the hood, there are mainly three costly steps:
- (1) (synchronize | data tracing | encode): make the data "JAX ready"
- (2) (compile | code tracing | execution): compile and optimize a function for XLA, execute it.
- (3) (format): convert data back to pandas/xarray/numpy format.

With the `wax.stream` primitives, it is quite easy to explicitly split the 1-step workflow
into a 3-step workflow.

This will allow the user to have full control over each step and iterate on each one.

It is actually very useful to iterate on step (2), the "calculation step" when
you are doing research.
You can then take full advantage of the JAX primitives, especially the `jit` primitive.

Let's illustrate how to reimplement WAX-ML EWMA yourself with the WAX-ML 3-step workflow.

## Imports

In [4]:

import haiku as hk
import numpy as onp
import pandas as pd
import xarray as xr

from wax.accessors import register_wax_accessors
from wax.compile import jit_init_apply
from wax.external.eagerpy import convert_to_tensors
from wax.format import format_dataframe
from wax.modules import EWMA
from wax.stream import tree_access_data
from wax.unroll import dynamic_unroll

register_wax_accessors()

## Performance on big dataframes

### Generate data

In [5]:
T = 1.0e5
N = 1000

In [6]:
%%time
T, N = map(int, (T, N))
dataframe = pd.DataFrame(
    onp.random.normal(size=(T, N)), index=pd.date_range("1970", periods=T, freq="s")
)

CPU times: user 2.46 s, sys: 225 ms, total: 2.68 s
Wall time: 2.68 s


### Pandas EWMA

In [7]:
%%time
df_ewma_pandas = dataframe.ewm(alpha=1.0 / 10.0).mean()

CPU times: user 2.86 s, sys: 465 ms, total: 3.33 s
Wall time: 3.33 s


### WAX-ML EWMA

In [8]:
%%time
df_ewma_wax = dataframe.wax.ewm(alpha=1.0 / 10.0).mean()

CPU times: user 2.09 s, sys: 374 ms, total: 2.46 s
Wall time: 2.42 s


It's a little faster, but not that much faster...

### WAX-ML EWMA (without format step)

Let's disable the final formatting step (the output is now in raw JAX format):

In [9]:
%%time
df_ewma_wax_no_format = dataframe.wax.ewm(alpha=1.0 / 10.0, format_outputs=False).mean()

CPU times: user 450 ms, sys: 244 ms, total: 694 ms
Wall time: 693 ms


In [34]:
type(df_ewma_wax_no_format)

jaxlib.xla_extension.DeviceArray

Let's check the device on which the calculation was performed (if you have GPU available, this should be `GpuDevice` otherwise it will be `CpuDevice`):

In [35]:
df_ewma_wax_no_format.device()

CpuDevice(id=0)

That's better! In fact (see below)
there is a performance problem in the final formatting step.
See WEP3 for a proposal to improve the formatting step.

### Generate data (in dataset format)

WAX-ML `Sream` object works on datasets.
So let's transform the `DataFrame` into a xarray `Dataset`:

In [10]:
dataset = xr.DataArray(dataframe).to_dataset(name="dataarray")

## Step (1) (synchronize | data tracing | encode)

In this step,  WAX-ML do:
- "data tracing" : prepare the indices for fast access tin the JAX function `access_data`
- synchronize streams if there is multiple ones.
  This functionality have options : `freq`, `ffills`
- encode and convert data from numpy to JAX: use encoders for `datetimes64` and `string_`
  dtypes. Be aware that by default JAX works in float32
  (see [JAX's Common Gotchas](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#double-64bit-precision) to work in float64).

In [11]:
%%time
stream = dataframe.wax.stream()
np_data, np_index, xs = stream.trace_dataset(dataset)
jnp_data, jnp_index, jxs = convert_to_tensors((np_data, np_index, xs), "jax")

CPU times: user 212 ms, sys: 2.62 ms, total: 215 ms
Wall time: 107 ms


/usr/local/anaconda3/envs/wax-env/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py:2977: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, "asarray")
/usr/local/anaconda3/envs/wax-env/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py:2977: UserWarning: Explicitly requested dtype int64 requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, "asarray")


In [12]:
from jax.tree_util import tree_leaves, tree_map

In [13]:
# We explicitly set data on CPUs (the is not needed if you only have CPUs)
cpus = jax.devices("cpu")
jnp_data, jnp_index, jxs = tree_map(
    lambda x: jax.device_put(x, cpus[0]), (jnp_data, jnp_index, jxs)
)
print("data copied to CPU device.")

data copied to CPU device.


We have now "JAX-ready" data for later fast access.

## Step (2) (compile | code tracing | execution)

In this step we:
- prepare a pure function (with
  [Haiku's transform mechanism](https://dm-haiku.readthedocs.io/en/latest/api.html#haiku-transforms))
  Define a "transformation" function which:
    - access to the data
    - apply another transformation, here: EWMA

- compile it with `jax.jit`
- perform code tracing and execution (the last line):
    - Unroll the transformation on "steps" `xs` (a `np.arange` vector).

In [14]:
%%time
@jit_init_apply
@hk.transform_with_state
def transform_dataset(step):
    dataset = tree_access_data(jnp_data, jnp_index, step)
    return EWMA(alpha=1.0 / 10.0, adjust=True)(dataset["dataarray"])


rng = next(hk.PRNGSequence(42))
outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

CPU times: user 1.78 s, sys: 269 ms, total: 2.05 s
Wall time: 1.93 s


In [15]:
outputs.device()

CpuDevice(id=0)

Once it has been compiled and "traced" by JAX, the function is much faster to execute:

In [16]:
%%timeit
outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

1.57 s ± 6.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%time
outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

CPU times: user 149 ms, sys: 128 ms, total: 277 ms
Wall time: 133 ms


This is 3x faster than pandas implementation!

(The 3x factor is obtained by measuring the execution with %timeit.
We don't know why, but when executing a code cell once at a time, then the execution time can vary a lot and we can observe some executions with a speed-up of 100x).

## Step(3) (format)
Let's come back to pandas/xarray:

In [19]:
%%time
y = format_dataframe(
    dataset.coords, onp.array(outputs), format_dims=dataset.dataarray.dims
)

CPU times: user 1.49 s, sys: 18.2 ms, total: 1.51 s
Wall time: 1.51 s


It's quite slow (see WEP3 enhancement proposal).

## GPU execution

## GPU execution

Let's look with execution on GPU

In [20]:
try:
    gpus = jax.devices("gpu")
    jnp_data, jnp_index, jxs = tree_map(
        lambda x: jax.device_put(x, gpus[0]), (jnp_data, jnp_index, jxs)
    )
    print("data copied to GPU device.")
    GPU_AVAILABLE = True
except RuntimeError as err:
    print(err)
    GPU_AVAILABLE = False

Unknown backend gpu. Available: ['interpreter', 'cpu']


Let's check that our data is on the GPUs:

In [21]:
tree_leaves(jnp_data)[0].device()

CpuDevice(id=0)

In [22]:
tree_leaves(jnp_index)[0].device()

CpuDevice(id=0)

In [23]:
jxs.device()

CpuDevice(id=0)

In [24]:
%%time
if GPU_AVAILABLE:
    rng = next(hk.PRNGSequence(42))
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.91 µs


Let's redefine our function `transform_dataset` by explicitly specify to `jax.jit` the `device` option.

In [25]:
%%time
if GPU_AVAILABLE:

    @hk.transform_with_state
    def transform_dataset(step):
        dataset = tree_access_data(jnp_data, jnp_index, step)
        return EWMA(alpha=1.0 / 10.0, adjust=True)(dataset["dataarray"])

    transform_dataset = type(transform_dataset)(
        transform_dataset.init, jax.jit(transform_dataset.apply, device=gpus[0])
    )

    rng = next(hk.PRNGSequence(42))
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.91 µs


In [26]:
outputs.device()

CpuDevice(id=0)

In [27]:
%%timeit
if GPU_AVAILABLE:
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

17.1 ns ± 0.0865 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
